# **NHẬN DIỆN KHUÔN MẶT REALTIME BẰNG FACENET & MTCNN:**

### **CHUẨN BỊ:**
để chuẩn bị sử dụng FaceNet và MTCNN thì các bạn cần phải đầy đủ các package này:

```
torch==1.8.0
torchvision==0.9.0
numpy==1.19.2
opencv-python==4.5.1.48
```

Sau khi đã đầy đủ các package trên thì tiến hành cài đặt thôi.
```
pip install facenet-pytorch
```

Vậy là hoàn thành và tiếp theo là phần nói về code và vận hành.

### **2. Face Detection:**
Sau đây là code mẫu dành cho Detection Face.
>_Nguồn: https://viblo.asia/p/nhan-dien-khuon-mat-voi-mang-mtcnn-va-facenet-phan-2-bJzKmrVXZ9N_

In [ ]:
import cv2
from facenet_pytorch import MTCNN
import torch

device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
mtcnn = MTCNN(thresholds= [0.7, 0.7, 0.8] ,keep_all=True, device = device)
cap = cv2.VideoCapture(0)   
# Set video frame width and height
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

# Read video frame-by-frame
while cap.isOpened():
    isSuccess, frame = cap.read()
    if isSuccess:
        boxes, _ = mtcnn.detect(frame)
        if boxes is not None:
            for box in boxes:
                bbox = list(map(int,box.tolist()))
                frame = cv2.rectangle(frame,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,225),3)
    cv2.imshow('Face detection', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()



Đoạn code ở đây thì đầu tiên là:
```python
# import các thư viện cần thiết
from facenet_pytorch import MTCNN
import torch
```
tiếp theo là sử dụng torch để kiểm tra xem có gpu không nếu có thì sử dụng để nhanh hơn nếu kh thì sử dụng cpu, 
```python
device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
```

Tiếp theo gọi một object từ class `MTCNN` ở đây `thresholds` chính là mức `thresholds` cho 3 lớp mạng P, R và O. `keep_all = True` là mình muốn nó detect các khuôn mặt đã nhận thấy hay không.
```python
mtcnn = MTCNN(thresholds= [0.7, 0.7, 0.8] ,keep_all=True, device = device)
```

Còn về phần để hiện được webcam realtime thì sử dụng `cap = cv2.VideoCapture(0)` để gọi webcam. có thể set coi frame hình mình mong muốn là gì với code này.
```python
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
```

Tiếp theo vòng lặp while if nó giúp chạy liên tục để nhận diện khuôn mặt ở đây MTCNN có cho mình một hàm `detect` giúp ta kiểm tra khuôn mặt trong frame hình và trả về tọa độ hộp chữ nhật bao quanh khuôn mặt `boxes, _ = mtcnn.detect(frame)` . Tiếp theo là nó sẽ tao cho mình một list box, mỗi box thì sẽ có `(bbox[0],bbox[1])` là cặp `(x,y)` sau đó thì gọi `frame = cv2.rectangle(frame,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,225),3)` để in ra hình chữ nhật gắn lên khi phát hiện mặt.
```python
# Read video frame-by-frame
while cap.isOpened():
    isSuccess, frame = cap.read()
    if isSuccess:
        boxes, _ = mtcnn.detect(frame)
        if boxes is not None:
            for box in boxes:
                bbox = list(map(int,box.tolist()))
                frame = cv2.rectangle(frame,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,225),3)
    cv2.imshow('Face detection', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break
```

### **3. Capture Face:**
Đơn giản thì Capture cũng không khác gì Dectect chỉ là không sử dụng hàm `dectect` mà dùng `mtcnn(frame)` để crop ảnh theo từng **`leap = 1`- bước nhảy** để trả về ảnh trong 1 folder. Class `MTCNN(margin = 30, keep_all=False, post_process=False, device = device)` cũng thay đổi một chút là thêm margin = 30 để có thêm cạnh viền khuôn mặt khi mà crop, `keep_all = False` cũng tắt để trả về khuôn mặt chính, chứ không trả về nhiều cái nhỏ, `post_process=False` để giữ nguyên tọa độ để lưu hình, còn True thì sẽ chuyển hóa ảnh liên tục cho FaceNet.
Sau đây là code mẫu: 
>_Nguồn: https://viblo.asia/p/nhan-dien-khuon-mat-voi-mang-mtcnn-va-facenet-phan-2-bJzKmrVXZ9N_

In [ ]:
import cv2
from facenet_pytorch import MTCNN
import torch
from datetime import datetime
import os

device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Cố định đường dẫn đến thư mục assets/images
script_dir = os.path.dirname(os.path.abspath(__file__))
IMG_PATH = os.path.join(script_dir, '..', 'assets', 'images') 
IMG_PATH = os.path.abspath(IMG_PATH)

# Tạo thư mục nếu chưa tồn tại assets/images/<username>
if not os.path.exists(IMG_PATH):
    os.makedirs(IMG_PATH)
    print(f"Created directory: {IMG_PATH}")

# Số lượng ảnh chụp cho người dùng là 20
count = 20
usr_name = input("Input ur name: ")
USR_PATH = os.path.join(IMG_PATH, usr_name)

# Tạo thư mục nếu chưa tồn tại assets/images/<username>
if not os.path.exists(USR_PATH):
    os.makedirs(USR_PATH)
    print(f"Created directory: {USR_PATH}")

leap = 1

# Khởi tạo MTCNN và mở camera
mtcnn = MTCNN(margin = 20, keep_all=False, post_process=False, device = device)
cap = cv2.VideoCapture(0)


if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

print(f"Capturing {count} images. Press ESC to stop.")
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
while cap.isOpened() and count:
    isSuccess, frame = cap.read()
    if mtcnn(frame) is not None and leap%2:
        path = os.path.join(USR_PATH, f'{timestamp}_img{21-count:02d}.jpg')
        face_img = mtcnn(frame, save_path = path)
        count-=1
    leap+=1
    cv2.imshow('Face Capturing', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

Giải thích đoạn code này một chút thì đầu tiên là device sẽ chọn cpu hoặc gpu, tiếp theo là mình sẽ xác minh đường dẫn cố định là assets/images/<username> nếu không có thì sẽ tự động tọa cho mình với đường dẫn cố định đó luôn.
```powershell
python -u "d:\Trifuu\Subjects\ComputerVision\Labs\Lab5\Trieu_FaceNet\srs\capture_face.py"
> cpu
> Created directory: d:\Trifuu\Subjects\ComputerVision\Labs\Lab5\Trieu_FaceNet\assets\images
> Input ur name: Trieu
> Created directory: d:\Trifuu\Subjects\ComputerVision\Labs\Lab5\Trieu_FaceNet\assets\images\Trieu
> Capturing 20 images. Press ESC to stop.
```
Tiếp theo là sẽ bắt đầu mở webcam lên phát hiện mặt và chụp theo từng frame hình số lẻ sau đó thì trong vòng lặp có một đoạn `path = os.path.join(USR_PATH, f'{timestamp}_img{21-count:02d}.jpg')` đoạn này sẽ dẫn mình đã đặt sẵn và cho tấm hình đó có đuôi ảnh là .jpg, với format là `datetime.now` là **năm-tháng-ngày-phút-giây** sau cùng thì có `str(count)` là số thứ tự ảnh ví dụ ở đây là `2021-07-03-22-39-22_img16.jpg` thì số 16 thứ tự của lần lấy.

> Có thể coi input ở **[đường dẫn này](../assets/images/Trieu)**